# $\rho$-VAE:

This is the basic implementaiton of the $rho$-VAE, a simple fix to the vanilla-VAE, which can be extended to other variants as a plug-and-play addition.

In [1]:
import numpy as np
from scipy.linalg import toeplitz
%precision  %.3f

import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
batch_size = 128
epochs = 10
seed = 1
log_interval = 10
z_dim = 5
database_name = 'mnist'
#database_name = 'cifar10'

In [3]:
torch.manual_seed(seed)
device = torch.device("cuda:1")
#device = torch.device("cpu")

kwargs = {'num_workers': 1, 'pin_memory': True}
if database_name.lower() == 'mnist':
    database_obj_train = datasets.MNIST('data/mnist', train=True, download=True,
                   transform=transforms.ToTensor())
    database_obj_test = datasets.MNIST('data/mnist', train=False, transform=transforms.ToTensor())

elif database_name.lower() == 'cifar10':
    database_obj_train = datasets.CIFAR10('data/cifar10', train=True, download=True,
                   transform=transforms.ToTensor())
    database_obj_test = datasets.CIFAR10('data/cifar10', train=False, transform=transforms.ToTensor())
    
    
train_loader = torch.utils.data.DataLoader(database_obj_train,
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(database_obj_test,
    batch_size=batch_size, shuffle=True, **kwargs)
try:
    N_train,image_x,image_y,num_channel = train_loader.dataset.data.shape
    data_dim = image_x * image_y * num_channel
except:
    N_train,image_x,image_y = train_loader.dataset.data.shape
    data_dim = image_x * image_y 
    num_channel = 1


100.1%

Extracting data/mnist/MNIST/raw/train-images-idx3-ubyte.gz


28.4%

0.5%

Extracting data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [8]:
class rho_VAE(nn.Module):
    def __init__(self):
        super(rho_VAE, self).__init__()

        self.fc1 = nn.Linear(data_dim, 400)
        self.fc21 = nn.Linear(400, z_dim)
        self.fc22 = nn.Linear(400, 1)  # rho 
        self.fc23 = nn.Linear(400, 1)  # s
        self.fc3 = nn.Linear(z_dim, 400)
        self.fc4 = nn.Linear(400, data_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), torch.tanh(self.fc22(h1)) , self.fc23(h1)    # -1<rho<1,and s>0

    def reparameterize(self, mu, rho, logs):

        z_q = torch.randn_like(rho).view(-1,1) * torch.sqrt(logs.exp())
        for j in range(1,z_dim):
            addenum = z_q[:,-1].view(-1,1)  + torch.randn_like(rho).view(-1,1) * torch.sqrt(logs.exp())
            z_q = torch.cat(( z_q, addenum ),1)        
        z_q  = z_q + mu  
        return z_q

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, rho, logs = self.encode(x.view(-1, data_dim))
        z = self.reparameterize(mu, rho, logs)
        return self.decode(z), mu, rho, logs

In [9]:
model = rho_VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [10]:
def loss_function(recon_x, x, mu, rho, logs):
    
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, data_dim), reduction='sum')
    ##
    KLD = 0.5 * ( torch.sum(mu.pow(2)) + - z_dim * logs - (z_dim - 1) * torch.log(1 - rho**2) +  z_dim * (logs.exp()-1)  )
    KLD = torch.mean(KLD)

    return BCE + KLD

In [11]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, rho, logs = model(data)
        loss = loss_function(recon_batch, data, mu, rho, logs)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [12]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, rho, logs = model(data)
            test_loss += loss_function(recon_batch, data, mu, rho, logs).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, num_channel, image_x, image_y)[:n]])
                save_image(comparison.cpu(),
                         './samples/rho_VAE/recon_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [13]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, z_dim).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, num_channel, image_x, image_y),'./samples/rho_VAE/sample_' + str(epoch) + '.png')



Train Epoch: 1 [0/60000 (0%)]	Loss: 561.397339
Train Epoch: 1 [1280/60000 (2%)]	Loss: 274.873108
Train Epoch: 1 [2560/60000 (4%)]	Loss: 233.343979
Train Epoch: 1 [3840/60000 (6%)]	Loss: 219.355774
Train Epoch: 1 [5120/60000 (9%)]	Loss: 215.697571
Train Epoch: 1 [6400/60000 (11%)]	Loss: 207.643616
Train Epoch: 1 [7680/60000 (13%)]	Loss: 209.528214
Train Epoch: 1 [8960/60000 (15%)]	Loss: 199.517151
Train Epoch: 1 [10240/60000 (17%)]	Loss: 193.269363
Train Epoch: 1 [11520/60000 (19%)]	Loss: 191.618332
Train Epoch: 1 [12800/60000 (21%)]	Loss: 186.109100
Train Epoch: 1 [14080/60000 (23%)]	Loss: 178.192001
Train Epoch: 1 [15360/60000 (26%)]	Loss: 184.432953
Train Epoch: 1 [16640/60000 (28%)]	Loss: 173.354126
Train Epoch: 1 [17920/60000 (30%)]	Loss: 171.046646
Train Epoch: 1 [19200/60000 (32%)]	Loss: 171.001495
Train Epoch: 1 [20480/60000 (34%)]	Loss: 169.813995
Train Epoch: 1 [21760/60000 (36%)]	Loss: 170.190720
Train Epoch: 1 [23040/60000 (38%)]	Loss: 161.346436
Train Epoch: 1 [24320/60000 